##IMPORTACIONES


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal.signaltools
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense,GRU,LSTM

from sklearn.preprocessing import MinMaxScaler
import random  # Importa el módulo random
import tensorflow as tf

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_absolute_error as MAE


import warnings
warnings.filterwarnings("ignore")


In [ ]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.9 MB/s eta 0:00:00


In [ ]:
from pmdarima import auto_arima

## SUAVIZACIONES EXPONENCIALES

In [ ]:
def predict_using_smoothexponential(df , n_output):

    sku = df['ID'].unique()

    pronosticos_smoothexponential = pd.DataFrame(columns=['ID', 'COD_SKU_SAP', 'UND_ORDEN', 'HWES2_ADD', 'HWES2_MUL', 'HWES3_MM', 'HWES3_MA', 'HWES3_AM', 'HWES3_AA'])

    for n in sku:

        nuevo = df[df['ID'] == n]
        total_reg = len(nuevo)

        train = nuevo[:total_reg-n_output]
        test = nuevo[total_reg-n_output:]


        model_HWES2_MUL = ExponentialSmoothing(train['UND_ORDEN'], trend='mul').fit(optimized=True)
        model_HWES2_ADD = ExponentialSmoothing(train['UND_ORDEN'], trend='add').fit(optimized=True)
        sm_exp3_mm = ExponentialSmoothing(train['UND_ORDEN'], trend='mul', seasonal='mul', seasonal_periods=12).fit(optimized=True)
        sm_exp3_ma = ExponentialSmoothing(train['UND_ORDEN'], trend='mul', seasonal='add', seasonal_periods=12).fit(optimized=True)
        sm_exp3_am = ExponentialSmoothing(train['UND_ORDEN'], trend='add', seasonal='mul', seasonal_periods=12).fit(optimized=True)
        sm_exp3_aa = ExponentialSmoothing(train['UND_ORDEN'], trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True)

        COD_SKU_SAP_value = test.loc[test['ID'] == n, 'COD_SKU_SAP'].tolist()
        und_orden = test.loc[test['ID'] == n, 'UND_ORDEN'].tolist()

        pronosticos_temp = pd.DataFrame({
            'ID': [n] * len(model_HWES2_ADD.forecast(n_output)),
            'COD_SKU_SAP': COD_SKU_SAP_value,
            'UND_ORDEN': und_orden,
            'HWES2_ADD': model_HWES2_ADD.forecast(n_output),
            'HWES2_MUL': model_HWES2_MUL.forecast(n_output),
            'HWES3_MM': pd.Series(sm_exp3_mm.forecast(n_output)),
            'HWES3_MA': pd.Series(sm_exp3_ma.forecast(n_output)),
            'HWES3_AM': pd.Series(sm_exp3_am.forecast(n_output)),
            'HWES3_AA': pd.Series(sm_exp3_aa.forecast(n_output))
        })
        pronosticos_smoothexponential = pronosticos_smoothexponential.append(pronosticos_temp, ignore_index=True)

    return pronosticos_smoothexponential



In [ ]:
pronosticos_smoothexponential

,ID,COD_SKU_SAP,UND_ORDEN,HWES2_ADD,HWES2_MUL,HWES3_MM,HWES3_MA,HWES3_AM,HWES3_AA
0,1,4,582,608.210149,616.587082,746.233350,684.494451,718.458378,730.820019
1,1,4,663,609.082716,626.196728,681.390113,644.428195,629.899204,579.830441
2,1,4,626,609.955283,635.956142,882.067050,645.486143,798.628657,671.489799
3,1,4,666,610.827849,645.867660,870.310037,543.984703,773.565561,613.489303
4,1,4,743,611.700416,655.933650,825.070694,642.234741,714.909227,606.962893
5,6,33,177,187.144363,184.522814,241.624185,210.382855,243.815052,212.183376
6,6,33,131,188.656426,184.388527,184.602134,191.705165,187.610989,193.944972
7,6,33,181,190.168490,184.254338,219.404545,211.906644,223.445306,214.603592
8,6,33,199,191.680554,184.120247,218.293920,211.426038,222.920892,214.565874
9,6,33,189,193.192617,183.986253,260.701133,233.494502,265.874349,236.984238


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Udemy/TESIS/datax_paper.csv', sep='|', encoding='latin-1')
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y')
df.set_index('FECHA', inplace=True)

n_output=5

pronosticos_smoothexponential = predict_using_smoothexponential(df , n_output)


In [ ]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_absolute_error as MAE

metric_error_MAE =  pd.DataFrame(columns=['ID','COD_SKU_SAP', 'HWES2_ADD','HWES2_MUL','HWES3_MM','HWES3_MA','HWES3_AM','HWES3_AA'])
metric_error_MSE =  pd.DataFrame(columns=['ID','COD_SKU_SAP', 'HWES2_ADD','HWES2_MUL','HWES3_MM','HWES3_MA','HWES3_AM','HWES3_AA'])
metric_error_MAPE = pd.DataFrame(columns=['ID','COD_SKU_SAP', 'HWES2_ADD','HWES2_MUL','HWES3_MM','HWES3_MA','HWES3_AM','HWES3_AA'])

sku = pronosticos_smoothexponential['ID'].unique()

for n in sku:
    nuevo = pronosticos_smoothexponential[pronosticos_smoothexponential['ID'] == n]


    test=nuevo["UND_ORDEN"]
    sm_exp2_add=nuevo['HWES2_ADD']
    sm_exp2_mul=nuevo['HWES2_MUL']
    sm_exp3_mm=nuevo['HWES3_MM']
    sm_exp3_ma=nuevo['HWES3_MA']
    sm_exp3_am=nuevo['HWES3_AM']
    sm_exp3_aa=nuevo['HWES3_AA']

    COD_SKU_SAP_value = nuevo.loc[nuevo['ID'] == n, 'COD_SKU_SAP'].unique()




    metric_error_MAPE_temp = pd.DataFrame({
        'ID': [n] ,
        'COD_SKU_SAP':COD_SKU_SAP_value,
        'HWES2_ADD': MAPE(test,sm_exp2_add),
        'HWES2_MUL': MAPE(test,sm_exp2_mul),

        'HWES3_MM': MAPE(test,sm_exp3_mm),
        'HWES3_MA': MAPE(test,sm_exp3_ma),
        'HWES3_AM': MAPE(test,sm_exp3_am),
        'HWES3_AA': MAPE(test,sm_exp3_aa)
    })
    metric_error_MAPE = metric_error_MAPE.append(metric_error_MAPE_temp, ignore_index=True)


# Obtener el valor mínimo entre las columnas 'metod_abc' y 'metod_zzz'

metric_error_MAPE

,ID,COD_SKU_SAP,HWES2_ADD,HWES2_MUL,HWES3_MM,HWES3_MA,HWES3_AM,HWES3_AA
0,1,4,0.082309,0.055651,0.227242,0.110815,0.151894,0.143151
1,6,33,0.121411,0.113866,0.292558,0.224124,0.314217,0.239407
2,45,475,0.082002,0.071209,0.056050,0.078007,0.062782,0.081404
3,46,485,0.133700,0.038167,0.072327,0.075525,0.076883,0.062827


In [ ]:
metric_error_MAPE.to_csv('/content/drive/MyDrive/Udemy/TESIS/MAPE_EXP.csv', sep='|', encoding='latin-1')


# SARIMAX ALL IN ONE

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Udemy/TESIS/datax_paper.csv', sep='|', encoding='latin-1')
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y')
df.set_index('FECHA', inplace=True)

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Lista para almacenar los resultados de la diferenciación
resultados_diferenciacion = []

# Iterar sobre cada SKU
for n, nuevo in df.groupby('ID'):
    COD_SKU_SAP_value = nuevo['COD_SKU_SAP'].unique()

    # Metodo 1: Diferencia
    nuevo['serie_estacionaria'] = nuevo['UND_ORDEN'].diff()
    serie_temporal = nuevo['serie_estacionaria'].dropna()

    # Aplicar la prueba de Dickey-Fuller
    resultado_adf = adfuller(serie_temporal)

    # Determinar si la serie es estacionaria
    estacionariedad = "Estacionaria" if resultado_adf[1] <= 0.05 else "NoEstacionaria"
    metodo_utilizado = "Diferencia"

    # Metodo 2: Doble Diferencia
    if estacionariedad == "NoEstacionaria":
        nuevo['serie_estacionaria'] = nuevo['UND_ORDEN'].diff().diff()
        serie_temporal = nuevo['serie_estacionaria'].dropna()
        resultado_adf = adfuller(serie_temporal)
        estacionariedad = "Estacionaria" if resultado_adf[1] <= 0.05 else "NoEstacionaria"
        metodo_utilizado = "DobleDiferencia"

  # Metodo 3: triple  Diferencia
    if estacionariedad == "NoEstacionaria":
        nuevo['serie_estacionaria'] = nuevo['UND_ORDEN'].diff().diff().diff()
        serie_temporal = nuevo['serie_estacionaria'].dropna()
        resultado_adf = adfuller(serie_temporal)
        estacionariedad = "Estacionaria" if resultado_adf[1] <= 0.05 else "NoEstacionaria"
        metodo_utilizado = "TripleDiferencia"

    # Agregar los resultados a la lista
    resultados_diferenciacion.append({
        'ID': n,
        'COD_SKU_SAP': COD_SKU_SAP_value[0],  # Usar el primer valor único
        'ESTACIONARIEDAD': estacionariedad,
        'METODO_UTILIZADO': metodo_utilizado,
        'RESULTADO_ADF': '{:.6f}'.format(resultado_adf[1])
    })
# Crear el DataFrame final
Diferenciacion = pd.DataFrame(resultados_diferenciacion)

# Agregar columna al DataFrame original indicando el tipo de método de diferenciación
df['TIPO_METODO_DIFERENCIACION'] = df.apply(lambda x: Diferenciacion.loc[Diferenciacion['ID'] == x['ID'], 'METODO_UTILIZADO'].values[0], axis=1)



In [ ]:
sku = df['ID'].unique()
pronosticos_sarima = pd.DataFrame(columns=['ID', 'COD_SKU_SAP', 'UND_ORDEN', 'AR', 'I', 'MA', 'SO_AR', 'SO_D', 'SO_MA', 'SARIMA'])

for n in sku:
    nuevo = df[df['ID'] == n]

    COD_SKU_SAP_value = nuevo.loc[nuevo['ID'] == n, 'COD_SKU_SAP'].unique()

    if 'Diferencia' in nuevo['TIPO_METODO_DIFERENCIACION'].values:
        nuevo['serie_estacionaria'] = nuevo['UND_ORDEN'].diff()

    if 'DobleDiferencia' in nuevo['TIPO_METODO_DIFERENCIACION'].values:
        nuevo['serie_estacionaria'] = nuevo['UND_ORDEN'].diff().diff()

    if 'TripleDiferencia' in nuevo['TIPO_METODO_DIFERENCIACION'].values:
        nuevo['serie_estacionaria'] = nuevo['UND_ORDEN'].diff().diff().diff()

    model_auto2 = auto_arima(nuevo.serie_estacionaria.dropna(), m=12, max_order=None, max_p=9, max_q=9, max_d=2, max_P=5, max_Q=5, max_D=3, maxiter=60, alpha=0.05, n_jobs=-1,
                             trend='ct', information_criterion='oob', out_of_sample_size=5)

    ar = model_auto2.order[0]
    i = model_auto2.order[1]
    ma = model_auto2.order[2]
    so_ar = model_auto2.seasonal_order[0]
    so_d = model_auto2.seasonal_order[1]
    so_ma = model_auto2.seasonal_order[2]

    model = SARIMAX(nuevo.serie_estacionaria[:-5].dropna(), order=(ar, i, ma), seasonal_order=(so_ar, so_d, so_ma, 12))
    result = model.fit()

    und_orden = df.loc[df['ID'] == n, 'UND_ORDEN'].tolist()
    fecha =nuevo.index.tolist()

    # Invertir la diferenciación según el tipo de método
    if 'Diferencia' in nuevo['TIPO_METODO_DIFERENCIACION'].values:
        result_forecast = und_orden[-1] + np.cumsum(result.forecast(steps=5))
    elif 'DobleDiferencia' in nuevo['TIPO_METODO_DIFERENCIACION'].values:
        result_forecast = und_orden[-1] + np.cumsum(np.cumsum(result.forecast(steps=5)))
    elif 'TripleDiferencia' in nuevo['TIPO_METODO_DIFERENCIACION'].values:
        result_forecast = und_orden[-1] + np.cumsum(np.cumsum(np.cumsum(result.forecast(steps=5))))


    # Agregar a DataFrame de pronósticos
    pronosticos_temp = pd.DataFrame({
        'ID': [n] * 5,
        'COD_SKU_SAP': COD_SKU_SAP_value[0],
        'UND_ORDEN': und_orden[-5:],  # Selecciona los últimos 5 valores de UND_ORDEN
        'FECHA': fecha[-5:],
        'AR': ar,
        'I': i,
        'MA': ma,
        'SO_AR': so_ar,
        'SO_D': so_d,
        'SO_MA': so_ma,
        'SARIMA': result_forecast
    })
    pronosticos_sarima = pronosticos_sarima.append(pronosticos_temp, ignore_index=True)

pronosticos_sarima

,ID,COD_SKU_SAP,UND_ORDEN,AR,I,MA,SO_AR,SO_D,SO_MA,SARIMA,FECHA
0,1,4,582,0,0,1,2,0,0,705.439088,2023-01-01
1,1,4,663,0,0,1,2,0,0,745.913949,2023-02-01
2,1,4,626,0,0,1,2,0,0,729.383135,2023-03-01
3,1,4,666,0,0,1,2,0,0,725.353904,2023-04-01
4,1,4,743,0,0,1,2,0,0,724.533592,2023-05-01
5,6,33,177,0,0,0,3,0,0,195.472135,2023-01-01
6,6,33,131,0,0,0,3,0,0,176.041427,2023-02-01
7,6,33,181,0,0,0,3,0,0,194.141980,2023-03-01
8,6,33,199,0,0,0,3,0,0,194.320435,2023-04-01
9,6,33,189,0,0,0,3,0,0,212.094820,2023-05-01


In [ ]:

metric_error_MAPE = pd.DataFrame(columns=['ID','COD_SKU_SAP', 'SARIMA_MAPE'])

sku = pronosticos_sarima['ID'].unique()

for n in sku:
    nuevo = pronosticos_sarima[pronosticos_sarima['ID'] == n]

    test=nuevo["UND_ORDEN"]
    sarima=nuevo['SARIMA']

    COD_SKU_SAP_value = nuevo.loc[nuevo['ID'] == n, 'COD_SKU_SAP'].unique()

    metric_error_MAPE_temp = pd.DataFrame({
        'ID': [n] ,
        'COD_SKU_SAP':COD_SKU_SAP_value,
        'SARIMA_MAPE': MAPE(test,sarima),
        'AR':nuevo['AR'].unique(),
        'I': nuevo['I'].unique(),
        'MA':nuevo['MA'].unique(),
        'SO_AR':nuevo['SO_AR'].unique(),
        'SO_D':nuevo['SO_D'].unique(),
        'SO_MA':nuevo['SO_MA'].unique()
    })
    metric_error_MAPE = metric_error_MAPE.append(metric_error_MAPE_temp, ignore_index=True)



#metric_error_MAPE = metric_error_MAPE.drop('COD_SKU_SAP', axis=1)

resumen_metricas_error_SARIMA =metric_error_MAPE

resumen_metricas_error_SARIMA['MET_SARIMA']='('+resumen_metricas_error_SARIMA['AR'].astype(str)  +','+ resumen_metricas_error_SARIMA['I'].astype(str)  +','+  resumen_metricas_error_SARIMA['MA'].astype(str) +')('+ resumen_metricas_error_SARIMA['SO_AR'].astype(str) +','+ resumen_metricas_error_SARIMA['SO_D'].astype(str) +','+ resumen_metricas_error_SARIMA['SO_MA'].astype(str) + ',12)'

resumen_metricas_error_SARIMA


,ID,COD_SKU_SAP,SARIMA_MAPE,AR,I,MA,SO_AR,SO_D,SO_MA,MET_SARIMA
0,1,4,0.123255,0,0,1,2,0,0,"(0,0,1)(2,0,0,12)"
1,6,33,0.133302,0,0,0,3,0,0,"(0,0,0)(3,0,0,12)"
2,45,475,0.499573,0,0,0,2,1,0,"(0,0,0)(2,1,0,12)"
3,46,485,0.072906,1,0,0,0,0,0,"(1,0,0)(0,0,0,12)"


In [ ]:
resumen_metricas_error_SARIMA.to_csv('/content/drive/MyDrive/Udemy/TESIS/MAPE_SARIMA.csv', sep='|', encoding='latin-1')


# REDES NEURONALES

## LSTM

In [ ]:

seed_value = 42

def predict_using_lstm(df, n_output, n_epochs):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    pronosticos_rn_lstm = pd.DataFrame(columns=['ID','COD_SKU_SAP', 'UND_ORDEN', 'LSTM'])

    sku_list =  df['ID'].unique()

    for n in sku_list:

        nuevo = df[df['ID'] == n]
        total_reg = len(nuevo)

        COD_SKU_SAP = nuevo[['COD_SKU_SAP']][total_reg - n_output:]
        ID = nuevo[['ID']][total_reg - n_output:]
        FECHA_ORDEN = nuevo.index[total_reg - n_output:]


        nuevo = nuevo[['UND_ORDEN']]
        train = nuevo[:total_reg - n_output]
        test = nuevo[total_reg - n_output:]

        scaler = MinMaxScaler()
        scaler.fit(train)

        scaled_train = scaler.transform(train)
        scaled_test = scaler.transform(test)

        generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_output, batch_size=1)
        X, y = generator[0]

        n_features = 1

        model = Sequential()
        model.add(LSTM(100, activation='relu', input_shape=(n_output, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        model.fit_generator(generator, epochs=n_epochs)

        first_eval_batch = scaled_train[-n_output:]
        first_eval_batch = first_eval_batch.reshape((1, n_output, n_features))
        predictions = []

        current_batch = first_eval_batch.copy()

        for i in range(len(test)):
            current_pred = model.predict(current_batch)[0]
            predictions.append(current_pred)
            current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

        true_predictions = scaler.inverse_transform(predictions)

        test['ID'] = ID
        test['COD_SKU_SAP'] = COD_SKU_SAP
        test['FECHA_ORDEN'] = FECHA_ORDEN
        test['LSTM'] = true_predictions


        pronosticos_rn_lstm = pd.concat([pronosticos_rn_lstm, test])

    pronosticos_rn_lstm.set_index('FECHA_ORDEN', inplace=True)  # Establece 'FECHA_ORDEN' como índice

    return pronosticos_rn_lstm

In [ ]:
data_path = "/content/drive/MyDrive/Udemy/TESIS/datax_paper.csv"
df = pd.read_csv(data_path, sep='|', encoding='latin-1')
df.set_index('FECHA', inplace=True)

n_output = 5
n_epochs = 100


pronosticos_rn_lstm = predict_using_lstm(df, n_output, n_epochs)

In [ ]:
pronosticos_rn_lstm

,ID,COD_SKU_SAP,UND_ORDEN,LSTM
FECHA_ORDEN,,,,
1/01/2023,1,4,582,664.901799
1/02/2023,1,4,663,680.270920
1/03/2023,1,4,626,699.098071
1/04/2023,1,4,666,708.359701
1/05/2023,1,4,743,709.618116
1/01/2023,6,33,177,171.519551
1/02/2023,6,33,131,170.076735
1/03/2023,6,33,181,171.855808
1/04/2023,6,33,199,172.414928


## GRU

In [ ]:

seed_value = 42

# ... importaciones ...

def predict_using_gru(df, n_input, n_epochs):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    pronosticos_rn_gru = pd.DataFrame(columns=['ID','COD_SKU_SAP', 'UND_ORDEN', 'GRU'])

    sku_list =  df['ID'].unique()

    for n in sku_list:

        nuevo = df[df['ID'] == n]
        total_reg = len(nuevo)
        ID = nuevo[['ID']][total_reg - n_input:]
        COD_SKU_SAP = nuevo[['COD_SKU_SAP']][total_reg - n_input:]
        FECHA_ORDEN = nuevo.index[total_reg - n_input:]

        nuevo = nuevo[['UND_ORDEN']]
        train = nuevo[:total_reg - n_input]
        test = nuevo[total_reg - n_input:]

        scaler = MinMaxScaler()
        scaler.fit(train)

        scaled_train = scaler.transform(train)
        scaled_test = scaler.transform(test)

        generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)
        X, y = generator[0]

        n_features = 1

        model = Sequential()
        model.add(GRU(100, activation='relu', input_shape=(n_input, n_features)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        model.fit_generator(generator, epochs=n_epochs)

        first_eval_batch = scaled_train[-n_input:]
        first_eval_batch = first_eval_batch.reshape((1, n_input, n_features))
        predictions = []

        current_batch = first_eval_batch.copy()

        for i in range(len(test)):
            current_pred = model.predict(current_batch)[0]
            predictions.append(current_pred)
            current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

        true_predictions = scaler.inverse_transform(predictions)

        test['ID'] = ID
        test['GRU'] = true_predictions
        test['COD_SKU_SAP'] = COD_SKU_SAP
        test['FECHA_ORDEN'] = FECHA_ORDEN

        pronosticos_rn_gru = pd.concat([pronosticos_rn_gru, test])

    pronosticos_rn_gru.set_index('FECHA_ORDEN', inplace=True)  # Establece 'FECHA_ORDEN' como índice

    return pronosticos_rn_gru

In [ ]:
data_path = "/content/drive/MyDrive/Udemy/TESIS/datax_paper.csv"
df = pd.read_csv(data_path, sep='|', encoding='latin-1')
df.set_index('FECHA', inplace=True)


n_input = 5
n_epochs = 100

pronosticos_rn_gru = predict_using_gru(df , n_input, n_epochs)

Epoch 1/100
79/79 [==============================] - 2s 5ms/step - loss: 0.0433
Epoch 2/100
79/79 [==============================] - 0s 5ms/step - loss: 0.0173
Epoch 3/100
79/79 [==============================] - 1s 8ms/step - loss: 0.0163
Epoch 4/100
79/79 [==============================] - 1s 9ms/step - loss: 0.0172
Epoch 5/100
79/79 [==============================] - 0s 6ms/step - loss: 0.0159
Epoch 6/100
79/79 [==============================] - 0s 5ms/step - loss: 0.0160
Epoch 7/100
79/79 [==============================] - 0s 6ms/step - loss: 0.0154
Epoch 8/100
79/79 [==============================] - 0s 6ms/step - loss: 0.0151
Epoch 9/100
79/79 [==============================] - 1s 11ms/step - loss: 0.0137
Epoch 10/100
79/79 [==============================] - 1s 8ms/step - loss: 0.0157
Epoch 11/100
79/79 [==============================] - 1s 7ms/step - loss: 0.0146
Epoch 12/100
79/79 [==============================] - 0s 6ms/step - loss: 0.0140
Epoch 13/100
79/79 [================

## ERROR

In [ ]:
predict_using_gru

<function __main__.predict_using_gru(df, n_input, n_epochs)>

In [ ]:

pronosticos_rn=pronosticos_rn_lstm
pronosticos_rn['GRU']=pronosticos_rn_gru['GRU']
#pronosticos_rn['ID']=pronosticos_df['COD_SKU_SAP']
pronosticos_rn


,ID,COD_SKU_SAP,UND_ORDEN,LSTM,GRU
FECHA_ORDEN,,,,,
1/01/2023,1,4,582,664.901799,662.035729
1/02/2023,1,4,663,680.270920,687.787468
1/03/2023,1,4,626,699.098071,713.989265
1/04/2023,1,4,666,708.359701,727.293958
1/05/2023,1,4,743,709.618116,731.346081
1/01/2023,6,33,177,171.519551,179.767321
1/02/2023,6,33,131,170.076735,178.591648
1/03/2023,6,33,181,171.855808,183.050083
1/04/2023,6,33,199,172.414928,182.665987


In [ ]:
pronosticos_rn

NameError: name 'pronosticos_rn' is not defined

In [ ]:



metric_error_MAPE = pd.DataFrame(columns=['ID','COD_SKU_SAP', 'LSTM','GRU'])

sku = pronosticos_rn['ID'].unique()

for n in sku:
    nuevo = pronosticos_rn[pronosticos_rn['ID'] == n]


    test=nuevo["UND_ORDEN"]

    rn_lstm=nuevo['LSTM']
    rn_gru=nuevo['GRU']

    COD_SKU_SAP_value = nuevo.loc[nuevo['ID'] == n, 'COD_SKU_SAP'].unique()





    metric_error_MAPE_temp = pd.DataFrame({
        'ID': [n] ,
        'COD_SKU_SAP':COD_SKU_SAP_value,
        'LSTM': MAPE(test,rn_lstm),
        'GRU': MAPE(test,rn_gru)
    })
    metric_error_MAPE = metric_error_MAPE.append(metric_error_MAPE_temp, ignore_index=True)


# Obtener el valor mínimo entre las columnas

metric_error_MAPE


,ID,COD_SKU_SAP,LSTM,GRU
0,1,2870,0.209389,0.245452
1,2,2894,0.175463,0.161365
2,3,2904,0.229369,0.170892
3,4,2917,0.187200,0.306732
4,5,2960,0.066213,0.067371
5,6,3034,0.182710,0.193884
6,7,3045,0.423789,0.432924
7,8,3063,0.292084,0.263730
8,9,3064,0.197221,0.239935
9,10,3099,4.125693,2.195830


In [ ]:
metric_error_MAPE.to_csv('/content/drive/MyDrive/Udemy/TESIS/MAPE_RRN.csv', sep='|', encoding='latin-1')

# UNION Y SELECCION OPTIMO

In [ ]:
union_MAPE_all= pd.merge(pd.merge(resumen_metricas_error_SMEXP[['COD_SKU_SAP','MET_SMEXP_MAPE','EXPSMOOTH_MAPE']], resumen_metricas_error_SARIMA[['COD_SKU_SAP','MET_SARIMA','SARIMA_MAPE']], on='COD_SKU_SAP', how='inner'),resumen_metricas_error_RN[['COD_SKU_SAP','MET_RN_MAPE','RN_MAPE']], on='COD_SKU_SAP' ,how='inner')

union_MAPE_all['METODO_OPTIMO']=''
union_MAPE_all[['EXPSMOOTH_MAPE', 'SARIMA_MAPE','RN_MAPE']].min(axis=1)

0     0.233142
1     0.047709
2     0.126649
3     0.257136
4     0.092632
5     0.206036
6     0.113460
7     0.118164
8     0.183131
9     0.268814
10    0.187038
11    0.120308
12    0.096745
13    0.135571
14    0.121915
15    0.055870
16    0.049054
17    0.141214
18    0.170036
19    0.156330
dtype: float64

In [ ]:
union_MAPE_all= pd.merge(pd.merge(resumen_metricas_error_SMEXP[['COD_SKU_SAP','MET_SMEXP_MAPE','EXPSMOOTH_MAPE']], resumen_metricas_error_SARIMA[['COD_SKU_SAP','MET_SARIMA','SARIMA_MAPE']], on='COD_SKU_SAP', how='inner'),resumen_metricas_error_RN[['COD_SKU_SAP','MET_RN_MAPE','RN_MAPE']], on='COD_SKU_SAP' ,how='inner')

union_MAPE_all['METODO_OPTIMO']=''
union_MAPE_all['VALOR_MIN_MAPE'] = union_MAPE_all[['EXPSMOOTH_MAPE', 'SARIMA_MAPE','RN_MAPE']].min(axis=1)



# Creamos una función que obtiene la letra correspondiente al valor mínimo de 'valor_smt' y 'valor_auto'
def obtener_letra_menor(row):
  valores = [row['EXPSMOOTH_MAPE'], row['SARIMA_MAPE'], row['RN_MAPE']]
  letras = [row['MET_SMEXP_MAPE'], row['MET_SARIMA'], row['MET_RN_MAPE']]
  min_valor_index = valores.index(min(valores))
  return letras[min_valor_index]


# Aplicamos la función a lo largo de las filas con axis=1
union_MAPE_all['METODO_OPTIMO'] = union_MAPE_all.apply(obtener_letra_menor, axis=1)

union_MAPE_all


,COD_SKU_SAP,MET_SMEXP_MAPE,EXPSMOOTH_MAPE,MET_SARIMA,SARIMA_MAPE,MET_RN_MAPE,RN_MAPE,METODO_OPTIMO,VALOR_MIN_MAPE
0,4,HWES3_MA,0.233142,"(1,0,0)(0,0,0,12)",0.255633,LSTM,0.419657,HWES3_MA,0.233142
1,6,HWES3_MA,0.073794,"(0,1,1)(0,0,0,12)",0.092710,LSTM,0.047709,LSTM,0.047709
2,18,HWES2_ADD,0.126649,"(0,1,1)(1,0,0,12)",0.221094,GRU,0.134892,HWES2_ADD,0.126649
3,31,HWES2_ADD,0.257136,"(0,1,0)(0,0,0,12)",0.488216,GRU,18.931259,HWES2_ADD,0.257136
4,33,HWES2_ADD,0.092632,"(0,1,0)(1,0,0,12)",0.101053,LSTM,0.094116,HWES2_ADD,0.092632
5,38,HWES2_MUL,0.350436,"(2,1,2)(1,0,1,12)",0.206036,GRU,0.402300,"(2,1,2)(1,0,1,12)",0.206036
6,42,HWES2_MUL,0.129711,"(0,1,0)(2,0,0,12)",0.113460,GRU,0.118010,"(0,1,0)(2,0,0,12)",0.113460
7,224,HWES3_AM,0.118164,"(3,0,0)(0,0,2,12)",0.326339,LSTM,0.511706,HWES3_AM,0.118164
8,228,HWES3_MM,0.183131,"(4,0,1)(2,0,0,12)",0.351833,GRU,0.529135,HWES3_MM,0.183131
9,262,HWES2_MUL,0.399778,"(3,0,0)(0,0,0,12)",0.268814,GRU,0.551440,"(3,0,0)(0,0,0,12)",0.268814


# GRAFICAS

In [4]:
!pip install graphviz



In [6]:
from graphviz import Digraph

# Crear un nuevo gráfico
dot = Digraph(comment='Diagrama de Ubicaciones')

# Datos de ejemplo (solo dos ubicaciones)
locations = [
    {"ID": "DC001", "altura": "10m", "largo": "30m", "ancho": "20m", "tipo": "Almacén", "bahia": "A1", "pasillo": "P1", "posicion": "001"},
    {"ID": "DC002", "altura": "12m", "largo": "35m", "ancho": "25m", "tipo": "Centro de Distribución", "bahia": "A2", "pasillo": "P2", "posicion": "002"}
]

# Añadir nodos con estilo de corchete
for loc in locations:
    label = f"""
    ID: {loc['ID']}
    Altura: {loc['altura']}
    Largo: {loc['largo']}
    Ancho: {loc['ancho']}
    Tipo: {loc['tipo']}
    Bahía: {loc['bahia']}
    Pasillo: {loc['pasillo']}
    Posición: {loc['posicion']}
    """
    dot.node(loc['ID'], label, shape='rect')

# Añadir una arista entre las ubicaciones
dot.edge('DC001', 'DC002', label='Conecta a')

# Guardar y renderizar el archivo
output_path = 'ubicaciones_diagrama'
dot.render(output_path, format='png', view=True)



'ubicaciones_diagrama.png'